# Processing the final dataset



In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

data_folder = "projects/amazon-forest-regrowth/assets"



In [7]:
# 1 = Amazonia
# 2 = Caatinga
# 3 = Cerrado
# 4 = Mata Atlantica
# 5 = Pampa
# 6 = Pantanal

# region = "countrywide"
# region = "amaz" #for Amazonia
region = "atl" #for Mata Atlantica
# region = "cerr" #for Cerrado


In [22]:

export_mature_biomass = False
export_distance = False

biomes_br = ee.FeatureCollection(f"{data_folder}/raw/biomes_br")

if region == "countrywide":
    # Countrywide
    region_shp = biomes_br.geometry().dissolve()
elif region == "amaz":
    # Amazonia = 1
    region_shp = biomes_br.filter(ee.Filter.eq("CD_Bioma", 1))
elif region == "cerr":
    region_shp = biomes_br.filter(ee.Filter.eq("CD_Bioma", 3))
elif region == "atl":
    region_shp = biomes_br.filter(ee.Filter.eq("CD_Bioma", 4))



In [34]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass_500m_countrywide").clip(region_shp)
mature_biomass_vector = mature_biomass.gt(1).reduceToVectors(geometry=region_shp, scale=500, maxPixels=1e13)

secondary = ee.Image(f"{data_folder}/age_agbd").select("age")
distances_image = mature_biomass_vector.distance(searchRadius = 500000).toInt().updateMask(secondary)

def get_nearest_biomass(feature):
    agbd_value = mature_biomass.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=feature.geometry(),
        scale=500,
        maxPixels=1e13
    )
    return feature.set(agbd_value)

nearest_biomass = distances_image.reduceToVectors(geometry=region_shp, scale=500, maxPixels=1e13).map(get_nearest_biomass)
# nearest_biomass_image = ee.Image().paint(nearest_biomass, "first")

map = geemap.Map()
# map.addLayer(mature_biomass, {"min": 0, "max":400, 'palette':['blue', 'red']}, "mature_biomass")
map.addLayer(distances_image, {"min": 0, "max":100000, 'palette':['blue', 'red']}, "dist")
map.addLayer(nearest_biomass_image, {"min": 0, "max":400, 'palette':['blue', 'red']}, "nearest_biomass")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [6]:
# age_agbd = ee.Image(f"{data_folder}/age_agbd").clip(region_shp)
mature_biomass = ee.Image(f"{data_folder}/mature_biomass_500m_amaz")
mature_biomass = mature_biomass.addBands(mature_biomass.pixelLonLat().float())#.clip(region_shp)

mat_fc = mature_biomass.sample(region_shp, scale = 500, numPixels = 10000)

# # Export an ee.FeatureCollection to Google Drive, adjusting the description based on the years
task = ee.batch.Export.table.toDrive(
    collection=mat_fc, description="mature_amazon_500m", fileFormat="CSV"
)
task.start()

In [11]:
# map = geemap.Map()
# # map.addLayer(age_agbd.select("age"), {}, "Age")
# map.addLayer(mature_biomass, {"min": 0, "max": 400, "palette": ["red"]}, "Mature Biomass")
# map.addLayer(mat_fc, {"min": 0, "max": 400, "palette": ["red"]}, "mat_fc")
# map.centerObject(region_shp)
# map

In [19]:
# age_agbd = ee.Image(f"{data_folder}/age_agbd").clip(region_shp)
mature_biomass = ee.Image(f"{data_folder}/mature_biomass_500m_amaz")#.clip(region_shp)

mat_fc = mature_biomass.sample(region_shp, scale = 500, numPixels = 10000, geometries = True)

near_mat_biomass = mat_fc.kriging(propertyName = "mature_biomass", shape = "gaussian",
                                  range = 2.25, sill = 2700, nugget = 500, 
                                  reducer=ee.Reducer.mean())


# # Export an ee.FeatureCollection to Google Drive, adjusting the description based on the years
task = ee.batch.Export.image.toAsset(
    image=near_mat_biomass, assetId = f"projects/amazon-forest-regrowth/assets/near_mat_biomass", \
          description="near_mat_biomass", scale=500, maxPixels=1e13
)
task.start()
# map = geemap.Map()
# # map.addLayer(age_agbd.select("age"), {}, "Age")
# map.addLayer(mature_biomass, {"min": 0, "max": 400, "palette": ["red"]}, "Mature Biomass")
# map.addLayer(near_mat_biomass, {"min": 0, "max": 400, "palette": ["red"]}, "near_mat_biomass")
# map.centerObject(region_shp)
# map


In [ ]:
mature_biomass_aggregated = ee.Image(f"{data_folder}/mature_biomass_500m_{region}")

distance = mature_biomass_aggregated.fastDistanceTransform().sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).rename("distance") \
    .float()

years = [0, 5, 10, 15]

for year in years:

    if (year == 0):
        land_use_years = ee.Image(f"{data_folder}/land_use")
        name = f"{region}_all"
    else:
        land_use_years = ee.Image(f"{data_folder}/land_use_{year}_years")
        name = f"{region}_{year}yr"

    age_agbd = ee.Image(f"{data_folder}/age_agbd")
    categorical = ee.Image(f"{data_folder}/categorical")
    yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
    yearly_SI = ee.Image(f"{data_folder}/yearly_SI")
    cwd = ee.Image(f"{data_folder}/raw/cwd_chave")
    sur_cover = ee.Image(f"{data_folder}/sur_cover").rename("sur_cover")
    one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask")
    # mat = ee.Image(f"{data_folder}/nearest_mat_biomass_1000m_{region}").rename("distance", "mature_biomass")

    combined_mask = one_hectare_mask.And(land_use_years.select("last_LU"))
        # .And(mat.select("distance"))

    unified_data = (
        age_agbd.addBands(
            [categorical, land_use_years, cwd, sur_cover, yearly_mean_prec, yearly_SI
        ]).updateMask(combined_mask)
    )

    unified_data_fc = unified_data.stratifiedSample(
        numPoints = 5000, classBand = 'biome', region = unified_data.geometry()
    )


if export_distance:
    export_clipped_image(distance, 1000, 'both', 'distance', prefix=f"{year}_yr_")



In [42]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass_30m_countrywide").clip(region_shp)

if export_mature_biomass:
    export_clipped_image(mature_biomass, 500, 'both', f"mature_biomass_500m_{region}")

In [18]:
mature_biomass_aggregated = ee.Image(f"{data_folder}/mature_biomass_500m_{region}")

age_agbd = ee.Image(f"{data_folder}/age_agbd")

distance = mature_biomass_aggregated.fastDistanceTransform().sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).rename("distance") \
    .updateMask(age_agbd.select("age")).float()

if export_distance:
    export_clipped_image(distance, 1000, 'both', 'distance')

In [30]:
distance = ee.Image(f"{data_folder}/distance_1000m_{region}")

export_task = ee.batch.Export.image.toDrive(
    image=distance,
    description='distance_1000m_countrywide',
    region = region_shp,
    scale=1000,
    maxPixels=1e13
)
export_task.start()

In [7]:
def export_csv(year):
    if (year == 0):
        land_use_years = ee.Image(f"{data_folder}/land_use")
        name = f"{region}_all"
    else:
        land_use_years = ee.Image(f"{data_folder}/land_use_{year}_years")
        name = f"{region}_{year}yr"

    age_agbd = ee.Image(f"{data_folder}/age_agbd")
    categorical = ee.Image(f"{data_folder}/categorical")
    yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
    yearly_SI = ee.Image(f"{data_folder}/yearly_SI")
    cwd = ee.Image(f"{data_folder}/raw/cwd_chave")
    sur_cover = ee.Image(f"{data_folder}/sur_cover").rename("sur_cover")
    one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask")
    # mat = ee.Image(f"{data_folder}/nearest_mat_biomass_1000m_{region}").rename("distance", "mature_biomass")
    
    combined_mask = one_hectare_mask.And(land_use_years.select("last_LU"))
        # .And(mat.select("distance"))

    unified_data = (
        age_agbd.addBands(
            [categorical, land_use_years, cwd, sur_cover, yearly_mean_prec, yearly_SI
        ]).updateMask(combined_mask)
    )

    unified_data_fc = unified_data.stratifiedSample(
        numPoints = 5000, classBand = 'biome', region = unified_data.geometry()
    )

    # Export an ee.FeatureCollection to Google Drive, adjusting the description based on the years
    task = ee.batch.Export.table.toDrive(
        collection=unified_data_fc, description=name
    )
    task.start()

export_csv(0)
export_csv(5)
export_csv(10)
export_csv(15)